In [ ]:
import pandas as pd 
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from xgboost import XGBRegressor
from sklearn import preprocessing
import numpy as log1p
from sklearn.preprocessing import PolynomialFeatures



In [ ]:
#standardization
df=pd.read_csv("../input/30days-folds/train_folds.csv")
df_test= pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission=pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if "cat"in col]
numerical_cols= [col for col in useful_features if col.startswith("cont")]
df_test= df_test[useful_features]

final_predictions=[]
scores=[]
for fold in range(5):
    xtrain= df[df.kfold != fold].reset_index(drop=True)
    xvalid= df[df.kfold== fold].reset_index(drop= True)
    xtest= df_test.copy()
    ytrain= xtrain.target
    yvalid= xvalid.target
    
    xtrain= xtrain[useful_features]
    xvalid= xvalid[useful_features]
    
    
    
    
    
    
    ordinal_encoder= preprocessing.OrdinalEncoder()
    xtrain[object_cols]= ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols]= ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols]= ordinal_encoder.transform(xtest[object_cols])
    
    scaler= preprocessing.StandardScaler()
    xtrain[numerical_cols]=scaler.fit_transform(xtrain[numerical_cols])
    xvalid[numerical_cols]= scaler.transform(xvalid[numerical_cols])
    xtest[numerical_cols]= scaler.transform(xtest[numerical_cols])
    
    
    
    
    model= XGBRegressor(random_state=fold,tree_method='gpu_hist',gpu_id=0, predictor="gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid= model.predict(xvalid)
    test_preds=model.predict(xtest)
    final_predictions.append(test_preds)
    rmse= mean_squared_error(yvalid, preds_valid)
    print(fold,rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))
    


In [ ]:
#log transformation
df=pd.read_csv("../input/30days-folds/train_folds.csv")
df_test= pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission=pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if "cat"in col]
numerical_cols= [col for col in useful_features if col.startswith("cont")]
df_test= df_test[useful_features]

for col in numerical_cols:
    df[col]= np.log1p(df[col])
    df_test[col]= np.log1p(df_test[col])

final_predictions=[]
scores=[]
for fold in range(5):
    xtrain= df[df.kfold != fold].reset_index(drop=True)
    xvalid= df[df.kfold== fold].reset_index(drop= True)
    xtest= df_test.copy()
    ytrain= xtrain.target
    yvalid= xvalid.target
    
    xtrain= xtrain[useful_features]
    xvalid= xvalid[useful_features]
    
    
    
    
    
    
    ordinal_encoder= preprocessing.OrdinalEncoder()
    xtrain[object_cols]= ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols]= ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols]= ordinal_encoder.transform(xtest[object_cols])
    
    scaler= preprocessing.StandardScaler()
    xtrain[numerical_cols]=scaler.fit_transform(xtrain[numerical_cols])
    xvalid[numerical_cols]= scaler.transform(xvalid[numerical_cols])
    xtest[numerical_cols]= scaler.transform(xtest[numerical_cols])
    
    
    
    
    model= XGBRegressor(random_state=fold,tree_method='gpu_hist',gpu_id=0, predictor="gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid= model.predict(xvalid)
    test_preds=model.predict(xtest)
    final_predictions.append(test_preds)
    rmse= mean_squared_error(yvalid, preds_valid)
    print(fold,rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))
    


In [ ]:
#polynimial features
df=pd.read_csv("../input/30days-folds/train_folds.csv")
df_test= pd.read_csv("../input/30-days-of-ml/test.csv")
sample_submission=pd.read_csv("../input/30-days-of-ml/sample_submission.csv")

useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if "cat"in col]
numerical_cols= [col for col in useful_features if col.startswith("cont")]
df_test= df_test[useful_features]

poly= preprocessing.PolynomialFeatures(degree=2,interaction_only= True,include_bias=True,)
train_poly= poly.fit_transform(df[numerical_cols])
train_poly= poly.fit_transform(df_test[numerical_cols])

df_poly=pd.DataFrame(train_poly,columns=[f"poly_{i}" for i in range(train_poly.shape[1])])
df_test_poly=pd.DataFrame(train_poly,columns=[f"poly_{i}" for i in range(train_poly.shape[1])])

df=pd.concat([df,df_poly],axis=1)
df_test=pd.concat([df_test,df_test_poly],axis=1)

useful_features=[c for c in df.columns if c not in("id","target","kfold")]
object_cols= [col for col in useful_features if "cat"in col]

df_test= df_test[useful_features]


final_predictions=[]
scores=[]
for fold in range(5):
    xtrain= df[df.kfold != fold].reset_index(drop=True)
    xvalid= df[df.kfold== fold].reset_index(drop= True)
    xtest= df_test.copy()
    ytrain= xtrain.target
    yvalid= xvalid.target
    
    xtrain= xtrain[useful_features]
    xvalid= xvalid[useful_features]
    
    
    
    
    
    
    ordinal_encoder= preprocessing.OrdinalEncoder()
    xtrain[object_cols]= ordinal_encoder.fit_transform(xtrain[object_cols])
    xvalid[object_cols]= ordinal_encoder.transform(xvalid[object_cols])
    xtest[object_cols]= ordinal_encoder.transform(xtest[object_cols])
    
    scaler= preprocessing.StandardScaler()
    xtrain[numerical_cols]=scaler.fit_transform(xtrain[numerical_cols])
    xvalid[numerical_cols]= scaler.transform(xvalid[numerical_cols])
    xtest[numerical_cols]= scaler.transform(xtest[numerical_cols])
    
    
    
    
    model= XGBRegressor(random_state=fold,tree_method='gpu_hist',gpu_id=0, predictor="gpu_predictor")
    model.fit(xtrain, ytrain)
    preds_valid= model.predict(xvalid)
    test_preds=model.predict(xtest)
    final_predictions.append(test_preds)
    rmse= mean_squared_error(yvalid, preds_valid)
    print(fold,rmse)
    scores.append(rmse)
    
print(np.mean(scores), np.std(scores))
    


In [ ]:
sample_submission.shape,

In [ ]:
final_predictions[0],

In [ ]:
sample_submission["target"] = final_predictions[0]

In [ ]:
sample_submission.to_csv('submission_alka.csv', index=False)